# Minio provider allows for a standardized method structure exposed

[Minio documentation](https://docs.min.io/docs/python-client-api-reference.html)

### Ensure minio is installed

Ensure importer is loaded

In [2]:
%run /jupyter/importer.py

Appending finder


Ensure minio module has been installed with pip and is accessible by python

In [5]:
from Libs import ModulesManagement
ModulesManagement.ensureInstalled("minio", r"minio.*")

Module installed.


### Load configuration



In [6]:
import Config
minioCfg = Config.minioCfg()

Config
None
after split: Config
path: 
['/jupyter']
/jupyter/Config.ipynb
after split: Config
path: 
['/jupyter']
/jupyter/Config.ipynb
importing Jupyter notebook from /jupyter/Config.ipynb
Modules found.
['pyyaml==3.13']
Module already installed.


In [7]:
import minio
from typing import Sequence, Callable
from minio import Minio
from minio.error import (ResponseError, BucketAlreadyOwnedByYou,
                         BucketAlreadyExists)
from Libs.Storage.Contracts import (folder_info, file_info)
import Libs.file_utils
import io

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

class minio_client(object):
    minioClient: Minio = {}
    def authenticate(self):
        self.minioClient = Minio(minioCfg.url,
                        access_key=minioCfg.access_key,
                        secret_key=minioCfg.secret_key,
                        secure=minioCfg.secure)
        return
    def list_buckets(self) -> Sequence[minio.definitions.Bucket]:
        return self.minioClient.list_buckets()
    def create_bucket(self, name: str):
        return self.minioClient.make_bucket(name)
    def remove_bucket(self, name: str):
        return self.minioClient.remove_bucket(name)
    def list_folders(self, bucketName, prefix: str = None) -> Sequence[folder_info]:
        objs = self.minioClient.list_objects_v2(bucketName, prefix=prefix, recursive=False)
        return [ folder_info(bucketName, o.object_name, o.last_modified, o.etag) for o in filter(lambda x : x.is_dir, objs) ]
    def list_files(self, bucketName, prefix: str = None, recursive: bool = False) -> Sequence[folder_info]:
        objs = self.minioClient.list_objects_v2(bucketName, prefix=prefix, recursive=recursive)
        return [ file_info(bucketName, o.object_name, o.last_modified, o.etag, o.size) for o in filter(lambda x : not x.is_dir, objs) ]
    def get_content(self, bucketName, filePath) -> io.BytesIO:
        obj = self.minioClient.get_object(bucketName, filePath)
        data = io.BytesIO()
        data.write(obj.read())
        data.seek(0)
        return data
    def set_content(self, bucketName: str, filePath: str, data: io.BytesIO):
        self.minioClient.put_object(bucket_name=bucketName, object_name=filePath, data=data, length=data.getbuffer().nbytes)
    def remove_content(self, bucketName: str, filePath: str):
        self.minioClient.remove_object(bucket_name=bucketName, object_name=filePath)
        
        

Libs.Storage.Contracts
['/jupyter/Libs/Storage']
after split: Contracts
path: 
['/jupyter/Libs/Storage']
/jupyter/Libs/Storage/Contracts.ipynb
after split: Contracts
path: 
['/jupyter/Libs/Storage']
/jupyter/Libs/Storage/Contracts.ipynb
importing Jupyter notebook from /jupyter/Libs/Storage/Contracts.ipynb
Libs.file_utils
['/jupyter/Libs']
after split: file_utils
path: 
['/jupyter/Libs']
/jupyter/Libs/file_utils.ipynb
after split: file_utils
path: 
['/jupyter/Libs']
/jupyter/Libs/file_utils.ipynb
importing Jupyter notebook from /jupyter/Libs/file_utils.ipynb


In [86]:
#mc = minio_client()
#mc.authenticate()

In [87]:
#test client bucket management
#if "test" not in [l.name for l in mc.list_buckets()]:
#    mc.create_bucket("test")
#lst = mc.list_buckets()
#lst = [l.name for l in lst]
#print(lst)
#if "test" in lst:
#    mc.remove_bucket("test")
#[l.name for l in mc.list_buckets()]


['argo-artifacts', 'test', 'testbucket']


['argo-artifacts', 'testbucket']

In [88]:
#test folder list
#mc.list_folders(bucketName="testbucket")

[testbucket|testfolder/]

In [91]:
#mc.list_files(bucketName="testbucket", recursive=True)

NameError: name 'sizeof_fmt' is not defined

In [90]:
#ctnt = mc.get_content(bucketName="testbucket", filePath="testfolder/inspection-aliments-contrevenants.csv")
#mc.set_content(bucketName="testbucket", filePath="testfolder/inspection-aliments-contrevenants.csv", data=ctnt)